In [1]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, Normalizer, StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import expr
from pyspark.sql.session import SparkSession
from pyspark.sql.types import BooleanType
from helpers.helper_functions import translate_to_file_string
import pandas as pd
import numpy as np

# for pretty printing
def printDf(sprkDF): 
    newdf = sprkDF.toPandas()
    from IPython.display import display, HTML
    return HTML(newdf.to_html())

In [2]:
inputFile = "hdfs:///data/mushrooms.csv"

In [3]:
spark = (SparkSession
       .builder
       .master("yarn")
       .appName("Mushroom/Classification")
       .getOrCreate())

In [4]:
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ",") \
       .csv(inputFile)

# String- Werte in numerische Werte umwandeln

In [5]:
labelIndexer        = StringIndexer().setInputCol("class").setOutputCol("label").fit(df)
#cap_shapeIndexer    = StringIndexer().setInputCol("cap-shape").setOutputCol("cap_shapeNUM").fit(df)
#cap_surfaceIndexer  = StringIndexer().setInputCol("cap-surface").setOutputCol("cap_surfaceNUM").fit(df)
#cap_colorIndexer    = StringIndexer().setInputCol("cap-color").setOutputCol("cap_colorNUM").fit(df)
#bruisesIndexer      = StringIndexer().setInputCol("bruises").setOutputCol("bruisesNUM").fit(df)
odorIndexer         = StringIndexer().setInputCol("odor").setOutputCol("odorNUM").fit(df)
#gill_attachmentIndexer  = StringIndexer().setInputCol("gill-attachment").setOutputCol("gill_attachmentNUM").fit(df)
#gill_spacingIndexer     = StringIndexer().setInputCol("gill-spacing").setOutputCol("gill_spacingNUM").fit(df)
gill_sizeIndexer        = StringIndexer().setInputCol("gill-size").setOutputCol("gill_sizeNUM").fit(df)
gill_colorIndexer       = StringIndexer().setInputCol("gill-color").setOutputCol("gill_colorNUM").fit(df)
#stalk_shapeIndexer      = StringIndexer().setInputCol("stalk-shape").setOutputCol("stalk_shapeNUM").fit(df)
stalk_rootIndexer       = StringIndexer().setInputCol("stalk-root").setOutputCol("stalk_rootNUM").fit(df)
#stalk_surface_above_ringIndexer  = StringIndexer().setInputCol("stalk-surface-above-ring").setOutputCol("stalk_surface_above_ringNUM").fit(df)
#stalk_surface_below_ringIndexer  = StringIndexer().setInputCol("stalk-surface-below-ring").setOutputCol("stalk_surface_below_ringNUM").fit(df)
#stalk_color_above_ringIndexer    = StringIndexer().setInputCol("stalk-color-above-ring").setOutputCol("stalk_color_above_ringNUM").fit(df)
#stalk_color_below_ringIndexer    = StringIndexer().setInputCol("stalk-color-below-ring").setOutputCol("stalk_color_below_ringNUM").fit(df)
#veil_typeIndexer            = StringIndexer().setInputCol("veil-type").setOutputCol("veil_typeNUM").fit(df)
#veil_colorIndexer           = StringIndexer().setInputCol("veil-color").setOutputCol("veil_colorNUM").fit(df)
#ring_numberIndexer          = StringIndexer().setInputCol("ring-number").setOutputCol("ring_numberNUM").fit(df)
ring_typeIndexer            = StringIndexer().setInputCol("ring-type").setOutputCol("ring_typeNUM").fit(df)
spore_print_colorIndexer    = StringIndexer().setInputCol("spore-print-color").setOutputCol("spore_print_colorNUM").fit(df)
populationIndexer           = StringIndexer().setInputCol("population").setOutputCol("populationNUM").fit(df)
#habitatIndexer              = StringIndexer().setInputCol("habitat").setOutputCol("habitatNUM").fit(df)

Augrund den Erkenntnissen aus dem Notebook Dataunderstanding wird der Feature Vektor auf 7 Merkmale reduziert

In [6]:
featureCols = df.columns.copy()
featureCols.remove("class")
featureCols.remove("cap-shape")
featureCols.remove("cap-surface")
featureCols.remove("cap-color")
featureCols.remove("odor")
featureCols.remove("bruises")
featureCols.remove("gill-attachment")
featureCols.remove("gill-spacing")
featureCols.remove("gill-size")
featureCols.remove("gill-color")
featureCols.remove("stalk-shape")
featureCols.remove("stalk-root")
featureCols.remove("stalk-surface-above-ring")
featureCols.remove("stalk-surface-below-ring")
featureCols.remove("stalk-color-above-ring")
featureCols.remove("stalk-color-below-ring")
featureCols.remove("veil-type")
featureCols.remove("veil-color")
featureCols.remove("ring-number")
featureCols.remove("ring-type")
featureCols.remove("spore-print-color")
featureCols.remove("population")
featureCols.remove("habitat")

#featureCols = featureCols + ["cap_shapeNUM","cap_surfaceNUM", "cap_colorNUM", "bruisesNUM", "odorNUM", "gill_attachmentNUM", "gill_spacingNUM", "gill_sizeNUM", "gill_colorNUM", "stalk_shapeNUM", "stalk_rootNUM", "stalk_surface_above_ringNUM", "stalk_surface_below_ringNUM", "stalk_color_above_ringNUM", "stalk_color_below_ringNUM", "veil_colorNUM", "ring_numberNUM", "ring_typeNUM", "spore_print_colorNUM", "populationNUM", "habitatNUM"]
featureCols = featureCols + ["odorNUM","gill_sizeNUM", "gill_colorNUM", "stalk_rootNUM", "ring_typeNUM", "spore_print_colorNUM", "populationNUM"]

print(featureCols)


['odorNUM', 'gill_sizeNUM', 'gill_colorNUM', 'stalk_rootNUM', 'ring_typeNUM', 'spore_print_colorNUM', 'populationNUM']


In [7]:
labeledData = labelIndexer.transform(df)

#labeledDataNUM = cap_shapeIndexer.transform(cap_surfaceIndexer.transform(cap_colorIndexer.transform(bruisesIndexer.transform(odorIndexer.transform(gill_attachmentIndexer.transform(gill_spacingIndexer.transform(gill_sizeIndexer.transform(gill_colorIndexer.transform(stalk_shapeIndexer.transform(stalk_rootIndexer.transform(stalk_surface_above_ringIndexer.transform(stalk_surface_below_ringIndexer.transform(stalk_color_above_ringIndexer.transform(stalk_color_below_ringIndexer.transform(veil_colorIndexer.transform(veil_typeIndexer.transform(ring_numberIndexer.transform(ring_typeIndexer.transform(spore_print_colorIndexer.transform(populationIndexer.transform(habitatIndexer.transform(labeledData))))))))))))))))))))))
labeledDataNUM = odorIndexer.transform(gill_sizeIndexer.transform(gill_colorIndexer.transform(stalk_rootIndexer.transform(ring_typeIndexer.transform(spore_print_colorIndexer.transform(populationIndexer.transform(labeledData)))))))
assembler =  VectorAssembler(outputCol="features", inputCols=featureCols)

In [8]:
splits = labeledDataNUM.randomSplit([0.8, 0.2 ], 12345)
training = splits[0]
test = splits[1]

# Random Forest Klassifizierung mit Crossvalidation

In [9]:
evaluatorRF = BinaryClassificationEvaluator(labelCol="label",rawPredictionCol="prediction", metricName="areaUnderROC")

In [10]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", impurity="gini", minInstancesPerNode=10, subsamplingRate=0.95, seed=12345)

In [11]:
pipeline = Pipeline(stages= [assembler, rf])

In [12]:
paramGrid = ParamGridBuilder().addGrid(rf.numTrees, [3, 5, 10])\
            .addGrid(rf.maxDepth,[2,3,5,7])\
            .build()

In [13]:
cvRF = CrossValidator(estimator=pipeline, evaluator=evaluatorRF, estimatorParamMaps=paramGrid, numFolds=5, parallelism=2)

In [14]:
rfModel = cvRF.fit(training)

## Ausgabe Ergebnisse Crossvalidation

In [15]:
treeModel = rfModel.bestModel.stages[1]
print("Best Params: \n", treeModel.explainParams())
print("Param Map: \n", treeModel.extractParamMap())

Best Params: 
 bootstrap: Whether bootstrap samples are used when building trees. (default: True)
cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/

In [16]:
predictionsRF = rfModel.transform(test)

# Test Error und Confusion Matrix ausgeben

In [17]:
accuracyRF = evaluatorRF.evaluate(predictionsRF)
print("Test Error", (1.0 - accuracyRF))

Test Error 0.0031969309462915874


In [18]:
from pyspark.mllib.evaluation import MulticlassMetrics
predictionAndLabelsRF = predictionsRF.select("prediction", "label").rdd.map(lambda p: [p[0], float(p[1])])
metricsRF = MulticlassMetrics(predictionAndLabelsRF)
confusionRF = metricsRF.confusionMatrix()
print("Confusion matrix: \n" , confusionRF)

Confusion matrix: 
 DenseMatrix([[840.,   0.],
             [  5., 777.]])


In [ ]:
spark.stop()